#### Compute the Classical Lamination Theory (CLT) and First-order Shear Deformation Theory (FSDT) stiffness matrices for given stacking sequence.

### Classical Lamination Theory

In [6]:
import numpy as np

def stiffness_matrices_CLT(material_properties, angles, thicknesses, stacking_sequence):
    """
    Compute the A, B, and D stiffness matrices for a given laminate using CLT.

    Parameters:
    - material_properties: Dictionary containing the material properties for a lamina.
                           e.g., {'E1': value, 'E2': value, 'v12': value, 'G12': value}
    - angles: List of angles (in degrees) representing the orientation of each lamina.
    - thicknesses: List of thickness values for each lamina.
    - stacking_sequence: List indicating the order of the laminas.

    Returns:
    - Matrix [A]: Extensional stiffness matrix relating the resultant in-plane forces to the in-plane strains
    - Matrix [B]: Coupling stiffness matrix coupling the force and moment terms to the midplane strains and midplane curvatures
    - Matrix [D]: Bending stiffness matrix relating the resultant moments to the plate curvatures
    """

    # Initialize the A, B, and D matrices to zero
    A = np.zeros((3, 3))
    B = np.zeros((3, 3))
    D = np.zeros((3, 3))

    E11 = material_properties['E11']
    E22 = material_properties['E22']
    v12 = material_properties['v12']
    v21 = v12 * (E22/E11)
    G12 = material_properties['G12']

    # The elasticity tensor Q
    Q11 = E11 / (1 - v12 * v21)
    Q12 = (v12 * E22)/(1-v12 * v21)
    Q22 = E22 / (1 - v12 * v21)
    Q66 = G12
    
    z_bottom = -sum(thicknesses) / 2  # Start from the bottom-most layer

    # Loop through each lamina in the stacking sequence
    for i in stacking_sequence:
        theta = angles[i]
        t = thicknesses[i]

        # Convert angle to radians
        theta_rad = np.radians(theta)
        cos = np.cos(theta_rad)
        sin = np.sin(theta_rad)
        
        Q11_bar = Q11 * cos ** 4 + 2 * (Q12 + 2 * Q66) * cos ** 2 * sin ** 2 + Q22 * sin ** 4

        Q12_bar = (Q11 + Q22 - 4 * Q66) * cos ** 2 * sin ** 2 + Q12 * (cos ** 4 + sin ** 4)

        Q22_bar = Q11 * sin ** 4 + 2 * (Q12 + 2 * Q66) * cos ** 2 * sin ** 2 + Q22 * cos ** 4

        Q16_bar = (Q11 - Q12 - 2 * Q66) * cos ** 3 * sin + (Q12 - Q22 + 2 * Q66) * cos * sin ** 3        

        Q26_bar = (Q11 - Q12 - 2 * Q66) * cos * sin ** 3 + (Q12 - Q22 + 2 * Q66) * cos ** 3 * sin

        Q66_bar = (Q11 + Q22 - 2 * Q12 - 2 * Q66) * cos ** 2 * sin ** 2 + Q66 * (cos ** 4 + sin ** 4)

        # The transformed reduced stiffness matrix
        Q_bar = np.array([[Q11_bar, Q12_bar, Q16_bar], [Q12_bar, Q22_bar, Q26_bar], [Q16_bar, Q26_bar, Q66_bar]])
        
        # Update the top position of the current lamina
        z_top = z_bottom + t
        # print(z_bottom, z_top)

        # Compute the contributions to the A, B, and D matrices
        A += Q_bar * (z_top - z_bottom)
        B += 1/2 * Q_bar * (z_top**2 - z_bottom**2)
        D += 1/3 * Q_bar * (z_top**3 - z_bottom**3)

        # Update the bottom position for the next lamina
        z_bottom = z_top

    return A, B, D

# Example usage:
material_properties = {'E11': 0.30000e+08, 'E22': 0.30000e+07, 'v12': 0.25, 'G12': 0.15000e+07}
orientations = [0, 90, 0, 90]  # Example orientations in degrees
thicknesses = [0.5000e-02, 0.5000e-02, 0.5000e-02, 0.5000e-02]  # Example thicknesses in mm
stacking_sequence = [0, 1, 2, 3]  # Example sequence

A, B, D = stiffness_matrices_CLT(material_properties, orientations, thicknesses, stacking_sequence)

print("The stiffness Matrix A is")
print(A)
print()
print("The stiffness Matrix B is")
print(B)
print()
print("The stiffness Matrix D is")
print(D)
print()

The stiffness Matrix A is
[[3.32075472e+05 1.50943396e+04 4.50577596e-13]
 [1.50943396e+04 3.32075472e+05 1.61861336e-11]
 [4.50577596e-13 1.61861336e-11 3.00000000e+04]]

The stiffness Matrix B is
[[-6.79245283e+02 -3.55271368e-15  1.12644399e-15]
 [-3.55271368e-15  6.79245283e+02  4.04653341e-14]
 [ 1.12644399e-15  4.04653341e-14  7.10542736e-15]]

The stiffness Matrix D is
[[1.10691824e+01 5.03144654e-01 1.50192532e-17]
 [5.03144654e-01 1.10691824e+01 5.39537788e-16]
 [1.50192532e-17 5.39537788e-16 1.00000000e+00]]



### First-order Shear Deformation Theory (FSDT)

In [12]:
import numpy as np

def stiffness_matrices_FSDT(material_properties, angles, thicknesses, stacking_sequence):
    """
    Compute the A, B, and D stiffness matrices for a given laminate using FSDT.

    Parameters:
    - material_properties: Dictionary containing the material properties for a lamina.
                           e.g., {'E11': value, 'E22': value, 'v12': value, 'G12': value, 'G23': value, 'G13': value}
    - angles: List of angles (in degrees) representing the orientation of each lamina.
    - thicknesses: List of thickness values for each lamina.
    - stacking_sequence: List indicating the order of the laminas.

    Returns:
    - Matrix [A]: Extensional stiffness matrix relating the resultant in-plane forces to the in-plane strains
    - Matrix [B]: Coupling stiffness matrix coupling the force and moment terms to the midplane strains and midplane curvatures
    - Matrix [D]: Bending stiffness matrix relating the resultant moments to the plate curvatures
    """

    # Initialize the A, B, and D matrices to zero
    A = np.zeros((3, 3))
    B = np.zeros((3, 3))
    D = np.zeros((3, 3))

    E11 = material_properties['E11']
    E22 = material_properties['E22']
    v12 = material_properties['v12']
    G12 = material_properties['G12']
    v21 = v12 * (E22/E11)

    Q11 = E11 / (1 - v12 * v21)
    Q12 = (v12 * E22) / (1 - v12 * v21)
    Q22 = E22 / (1 - v12 * v21)
    Q66 = G12

    z_bottom = -sum(thicknesses) / 2  # Start from the bottom-most layer

    # Loop through each lamina in the stacking sequence
    for i in stacking_sequence:
        theta = angles[i]
        t = thicknesses[i]

        # Convert angle to radians
        theta_rad = np.radians(theta)
        cos = np.cos(theta_rad)
        sin = np.sin(theta_rad)
        
        # Transformation matrix
        T = np.array([
            [cos**2, sin**2, 2*sin*cos],
            [sin**2, cos**2, -2*sin*cos],
            [-sin*cos, sin*cos, cos**2 - sin**2]
        ])
        Q = np.array([[Q11, Q12, 0], [Q12, Q22, 0], [0, 0, Q66]])
        Q_bar = np.linalg.inv(T) @ Q @ T

        # Update the top position of the current lamina
        z_top = z_bottom + t

        # Compute the contributions to the A, B, and D matrices
        A += Q_bar * t
        B += Q_bar * (z_top**2 - z_bottom**2) / 2
        D += Q_bar * (z_top**3 - z_bottom**3) / 3

        # Update the bottom position for the next lamina
        z_bottom = z_top

    return A, B, D


# Example usage:
material_properties = {'E11': 0.30000e+08, 'E22': 0.30000e+07, 'v12': 0.25, 'G12': 0.15000e+07, 'G13': 0.15000e+07, 'G23': 0.15000e+07}
orientations = [0, 90, 90, 0]  # Example orientations in degrees
thicknesses = [0.5000e-02, 0.5000e-02, 0.5000e-02, 0.5000e-02]  # Example thicknesses in mm
stacking_sequence = [0, 1, 2, 3]  # Example sequence

A, B, D = stiffness_matrices_FSDT(material_properties, orientations, thicknesses, stacking_sequence)

print("The stiffness Matrix A is")
print(A)
print()
print("The stiffness Matrix B is")
print(B)
print()
print("The stiffness Matrix D is")
print(D)
print()

The stiffness Matrix A is
[[ 3.32075472e+05  1.50943396e+04 -9.35815007e-13]
 [ 1.50943396e+04  3.32075472e+05  3.42092375e-11]
 [-4.67907503e-13  1.71046187e-11  3.00000000e+04]]

The stiffness Matrix B is
[[ 0.00000000e+00 -3.55271368e-15  0.00000000e+00]
 [-3.55271368e-15 -1.42108547e-14  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  7.10542736e-15]]

The stiffness Matrix D is
[[ 1.78616352e+01  5.03144654e-01 -7.79845839e-18]
 [ 5.03144654e-01  4.27672956e+00  2.85076979e-16]
 [-3.89922920e-18  1.42538489e-16  1.00000000e+00]]

